# Grafos (parte 2)

### Aplicação prática de grafos: Encontrando o menor caminho

Uma das aplicações mais úteis de grafos é encontrar o menor caminho entre dois pontos. **single-source shortest path (SSSP)**

#### 1\. Menor caminho em passos (grafos sem peso)

Se todas as conexões têm o mesmo "custo" (ou seja, não têm peso), o menor caminho é simplesmente o caminho com o **menor número de arestas**.

  - **Quem resolve?** A **Busca em Largura (BFS)** é perfeita para isso\!
  - **Exemplo Prático:** "Qual o menor número de conexões para ir do seu perfil no Instagram até o perfil de uma celebridade?". A BFS encontra a resposta, pois ela explora em camadas de distância. A primeira vez que ela encontrar a celebridade, terá sido pelo caminho com o menor número de "pulos".

In [ ]:
class GrafoLA:
    def __init__(self):
        # Usaremos um dicionário para mapear rótulos de nós (inteiros ou strings)
        # para suas listas de adjacência.
        self.adj = {}
        # Para controlar os nós visitados em algoritmos como Busca em Profundidade (DFS) e Busca em Largura (BFS),
        # usaremos um dicionário que mapeia o rótulo do nó para um status (0 para não visitado, 1 para visitado).
        self.__visitado = {}
        self.ordem_visitados = []

    def __str__(self):
        # Retorna uma representação legível do grafo.
        # Ex: "A: ['B', 'C']\nB: ['A']"
        return '\n'.join([f"{no}: {vizinhos}" for no, vizinhos in self.adj.items()])

    def adicionar_aresta(self, n1, n2, bidirecional=True):
        # Garante que os nós existam no dicionário de adjacência
        if n1 not in self.adj:
            self.adj[n1] = []
        if n2 not in self.adj:
            self.adj[n2] = []

        # Adiciona a aresta de n1 para n2
        self.adj[n1].append(n2)

        # Se for bidirecional, adiciona a aresta de n2 para n1
        if bidirecional:
            self.adj[n2].append(n1)

    def obter_vizinhos(self, n):
        # Retorna a lista de vizinhos de um nó.
        # Retorna uma lista vazia se o nó não existir.
        return self.adj.get(n, [])

    # Comum para DFS e BFS
    def __visitar(self, no): # 'node' -> 'no'
        # Marca um nó como visitado e o adiciona à ordem de visita.
        self.__visitado[no] = 1
        self.ordem_visitados.append(no)

    # DFS - Busca em Profundidade
    def __dfs(self, n):
        # Implementação recursiva da Busca em Profundidade (DFS).
        self.__visitar(n)
        for vizinho in self.obter_vizinhos(n):
            # Verifica se o vizinho não foi visitado.
            if self.__visitado.get(vizinho, 0) == 0:
                self.__dfs(vizinho)

    def DFS(self, no_inicial):
        # Inicia a Busca em Profundidade (DFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós para cada nova execução de DFS.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        if no_inicial in self.adj:
            self.__dfs(no_inicial)
        return self.ordem_visitados
    # DFS - Busca em profundidade

    # BFS - Busca em largura
    def BFS(self, no_inicial):
        # Inicia a Busca em Largura (BFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        fila = []  # Usaremos uma lista como fila para a BFS

        if no_inicial not in self.adj:
            return self.ordem_visitados

        self.__visitar(no_inicial)
        fila.append(no_inicial)

        while fila:
            no_atual = fila.pop(0)  # Pega o primeiro nó da fila
            for vizinho in self.obter_vizinhos(no_atual):
                if self.__visitado.get(vizinho, 0) == 0:
                    self.__visitar(vizinho)
                    fila.append(vizinho)
        return self.ordem_visitados
    # BFS - Busca em largura

    # Algoritmo para encontrar o menor caminho de um nó determinado para todos os outros, em grafos sem pesos
    def __sssp(self, no_inicial, limite, funcao_juncao):
        """
        Função auxiliar para encontrar o menor caminho usando uma abordagem similar a BFS.
        Adaptada para usar rótulos de nós arbitrários.
        """
        caminhos = {no_inicial: [no_inicial]} 
        fila = [(no_inicial, 0)] # (no, nível)
        
        while fila:
            no_atual, nivel = fila.pop(0)

            if limite is not None and nivel >= limite:
                continue

            for vizinho in self.obter_vizinhos(no_atual):
                if vizinho not in caminhos:
                    caminhos[vizinho] = funcao_juncao(caminhos[no_atual], [vizinho])
                    fila.append((vizinho, nivel + 1))
        return caminhos
    
    def SSSP(self, no_inicial, limite=None):
        """
        Encontra os menores caminhos de um nó de partida para todos os nós alcançáveis.
        """
        def funcao_juncao_padrao(caminho1, caminho2):
            return caminho1 + caminho2

        if no_inicial not in self.adj:
            return {} # Retorna um dicionário vazio se o nó de partida não existe

        return self.__sssp(no_inicial, limite, funcao_juncao_padrao)
    # Algoritmo para encontrar o menor caminho de um nó determinado para todos os outros, em grafos sem pesos

In [ ]:
# FUNÇÃO __sssp(self, no_inicial, limite, funcao_juncao):
#   """
#   Função auxiliar para encontrar o menor caminho usando uma abordagem similar a BFS.
#   Adaptada para usar rótulos de nós arbitrários.
#   """
#   DEFINIR caminhos = UM DICIONÁRIO ONDE no_inicial TEM O VALOR DE UMA LISTA CONTENDO no_inicial
#   DEFINIR fila = UMA LISTA CONTENDO UMA TUPLA (no_inicial, 0) // (no, nível)

#   ENQUANTO fila NÃO ESTÁ VAZIA:
#     DEFINIR no_atual, nivel = REMOVER PRIMEIRO ELEMENTO DA fila

#     SE limite NÃO É NULO E nivel É MAIOR OU IGUAL A limite:
#       CONTINUAR // Pula para a próxima iteração do loop
#     FIM SE

#     PARA CADA vizinho EM self.obter_vizinhos(no_atual):
#       SE vizinho NÃO ESTÁ EM caminhos:
#         DEFINIR caminhos[vizinho] = CHAMAR funcao_juncao(caminhos[no_atual], [vizinho])
#         ADICIONAR TUPLA (vizinho, nivel + 1) À fila
#       FIM SE
#     FIM PARA
#   FIM ENQUANTO

#   RETORNAR caminhos
# FIM FUNÇÃO

# ---

# FUNÇÃO SSSP(self, no_inicial, limite=NULO):
#   """
#   Encontra os menores caminhos de um nó de partida para todos os nós alcançáveis.
#   """
#   FUNÇÃO funcao_juncao_padrao(caminho1, caminho2):
#     RETORNAR caminho1 CONCATENADO COM caminho2
#   FIM FUNÇÃO

#   SE no_inicial NÃO ESTÁ EM self.adj:
#     RETORNAR UM DICIONÁRIO VAZIO // O nó de partida não existe
#   FIM SE

#   RETORNAR CHAMAR self.__sssp(no_inicial, limite, funcao_juncao_padrao)
# FIM FUNÇÃO

Imagine que chegamos no centro do labirinto, e agora queremos retornar para uma das saídas do labirinto (sim, o labirinto tem várias entradas diferentes). E existe mais de um caminho para sair desse labirinto.

Queremos sair dele o mais rápido possível, então precisamos sair pelo **caminho mais curto possível**, ou seja, o que vai nos fazer andar menos. Mas qual seria esse caminho?

A estratégia do SSSP é bem parecida com a Busca em Largura (BFS) que acabamos de ver, porque a BFS naturalmente encontra os caminhos mais curtos em um grafo não ponderado (que não tem peso nas arestas).

A função `SSSP` não só visita os "cantos" (nós) como a BFS comum, mas também **guarda qual foi o trajeto mais rápido** para chegar a cada um deles a partir do seu ponto de partida. Ela faz isso "camada por camada", garantindo que, quando encontra um nó pela primeira vez, o caminho que usou para chegar até ele é o menor possível.

#### As ferramentas extras:

Além das "ferramentas" da BFS (`self.adj`, `self.__visitado`, `fila`),

- `self.adj` **(Suas adjacências/vizinhos)**: O mapa do labirinto, mostrando quais cantos (nós) estão conectados diretamente.

- `self.__visitado` **(Seu diário de bordo)**: O diário onde você anota por quais cantos (nós) você já passou, para não revisitar.

- `self.ordem_visitados` **(Seu caminho percorrido)**: Uma lista que guarda a sequência de cantos (nós) que você visitou.

- `fila` **(Sua Lista de Espera)**: Esta é a ferramenta mais importante da BFS. É uma lista onde você coloca os cantos que você **descobriu, mas que ainda não explorou**. Quando você termina de explorar um canto, você pega o próximo da fila.

o SSSP utiliza também as seguintes:

- `caminhos` **(Seu mapa de rotas)**: Este é um dicionário especial. Para cada canto (nó) que você visita, ele anota qual é o **melhor caminho** (a sequência de nós) que você seguiu para chegar lá a partir do `no_inicial`.

- `limite` **(Sua distância máxima)**: Às vezes, você não quer saber o caminho para todos os lugares, mas só para aqueles que estão a uma certa distância máxima de você. O `limite` serve para isso: se um caminho ficar mais longo que o limite, ele não continua a explorar.

- `funcao_juncao` **(Seu guia para montar caminhos)**: Esta é uma pequena função que ajuda a "construir" os caminhos. Ela pega o caminho que você já tem até um certo ponto e simplesmente adiciona o próximo nó a ele, criando o caminho completo.

**Entendendo as funções: Passo a passo**

1. `__sssp(self, no_inicial, limite, funcao_juncao)`: A Navegação Detalhada**

    Esta é a função "auxiliar" que faz o trabalho pesado de encontrar os caminhos:

- **Preparação**:

    - `caminhos = {no_inicial: [no_inicial]} `: Você cria seu "mapa de rotas". O primeiro caminho que você conhece é o de você para você mesmo: `[no_inicial]`.

    - `fila = [(no_inicial, 0)] # (no, nível)`: Assim como na BFS, você tem uma fila. Mas agora, para cada item na fila, você guarda não só o nó, mas também o nível dele (a distância em "passos" do `no_inicial`). O `no_inicial` está no nível 0.

- **Exploração Contínua**:

    - `while fila:`: Enquanto houver lugares para explorar na fila:

    - `no_atual, nivel = fila.pop(0)`: Você pega o próximo nó da fila e o nível em que ele está. Lembre-se, a fila garante que você sempre explore os nós mais próximos primeiro!

    - `if limite is not None and nivel >= limite:`: Se você definiu um limite de distância e já alcançou ou ultrapassou esse limite, não faz sentido ir mais longe por esse caminho. Então, você **continua** para o próximo item da fila, ignorando o resto deste loop.

    - `for vizinho in self.obter_vizinhos(no_atual):`: Você olha para todos os vizinhos diretos do `no_atual`.

        - `if vizinho not in caminhos:`: Esta é a parte crucial! Se você **ainda não encontrou um caminho** para esse `vizinho` (ou seja, ele não está no seu `mapa de rotas`), significa que esta é a **primeira vez** que você está chegando a ele. Como a BFS explora em largura, essa primeira vez é garantidamente pelo caminho mais curto!

        - `caminhos[vizinho] = funcao_juncao(caminhos[no_atual], [vizinho])`: Você constrói o caminho para o `vizinho`. Pega o caminho que já tinha até o `no_atual` e simplesmente adiciona o `vizinho` ao final. Isso cria a rota completa.

        - `fila.append((vizinho, nivel + 1))`: Adiciona esse novo `vizinho` à fila para ser explorado mais tarde, e marca que ele está no próximo nível (um passo a mais que o `no_atual`).

- **Fim da Descoberta**:

    - `return caminhos`: Quando a fila está vazia, significa que você explorou todos os caminhos possíveis (dentro do seu limite, se houver). A função retorna o seu `mapa de rotas` completo, mostrando o caminho mais curto para cada nó alcançável.

**Entendendo a função `SSSP(self, no_inicial, limite=None)`: O Ponto de entrada**

    Esta é a função que você chama para começar a encontrar os caminhos:

- `def funcao_juncao_padrao(caminho1, caminho2): return caminho1 + caminho2`: Ela define como os pedacinhos de caminhos devem ser unidos. Basicamente, ela pega duas listas (o caminho até o nó anterior e o nó atual em uma lista) e as junta.

- `if no_inicial not in self.adj: return {}`: Novamente, uma verificação de segurança. Se o seu ponto de partida não existe no mapa do grafo, não há caminhos para encontrar, e a função retorna um dicionário vazio.

- `return self.__sssp(no_inicial, limite, funcao_juncao_padrao)`: Finalmente, ela chama a função auxiliar `__sssp` com o nó de partida, o limite opcional e a função que sabe juntar os caminhos.

----------------

Em resumo, a função `SSSP` é como ter um GPS que, a partir de onde você está, calcula a rota mais rápida para todos os outros destinos alcançáveis. Ela faz isso explorando a rede de forma organizada (em largura), garantindo que a primeira vez que ela "descobre" um destino, essa descoberta foi feita pela rota mais curta. Isso a torna muito útil para navegação, jogos e análise de redes sociais, por exemplo!

In [ ]:
g3 = GrafoLA()
g3.adicionar_aresta(0, 1)
g3.adicionar_aresta(0, 2)
g3.adicionar_aresta(1, 2)
g3.adicionar_aresta(1, 3)
g3.adicionar_aresta(2, 4)
g3.adicionar_aresta(3, 4)
g3.adicionar_aresta(4, 5)
g3.adicionar_aresta(5, 6)
print(g3, "\n")

print(g3.SSSP(no_inicial=0))

In [ ]:
g4 = GrafoLA()
g4.adicionar_aresta("Pedro", "João")
g4.adicionar_aresta("João", "Lucas")
g4.adicionar_aresta("Lucas", "Ricardo")
g4.adicionar_aresta("Ana", "Marcela")
g4.adicionar_aresta("Ana", "Ricardo")
print(g4, "\n")

print(g4.menor_caminho("Marcela"))


#### 2\. Menor caminho por custo (Grafos com peso)

E se as conexões tiverem custos diferentes (distância, tempo, preço)? O caminho com menos arestas pode não ser o mais barato ou o mais rápido.

  * **Exemplo prático:** O caminho de avião com menos escalas (ex: 1 escala) pode ser muito mais caro do que um caminho com 2 escalas. O Waze não te mostra o caminho com menos ruas, mas sim o mais **rápido**.

Nesses casos, a BFS não é suficiente. Precisamos de algoritmos mais avançados (como o Algoritmo de Dijkstra) que levem em conta o peso de cada aresta para encontrar o verdadeiro menor caminho.

A figura abaixo mostra um grafo com pesos nas aretas.


<img src=https://objectstorage.us-ashburn-1.oraclecloud.com/n/ida8x1uljntv/b/graph_images_ada/o/grafo_com_peso.png
 width=600>

A classe GrafoLA abaixo já suporta grafos ponderados.

In [ ]:
class GrafoLA:
    def __init__(self):
        # self.adj agora mapeia rótulos de nós (inteiros ou strings)
        # para uma lista de tuplas (vizinho, peso).
        # Ex: {'A': [('B', 5), ('C', 2)], 'B': [('A', 5)]}
        self.adj = {}
        
        # Para controlar os nós visitados em algoritmos como Busca em Profundidade (DFS) e Busca em Largura (BFS),
        # usaremos um dicionário que mapeia o rótulo do nó para um status (0 para não visitado, 1 para visitado).
        self.__visitado = {}
        self.ordem_visitados = []

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        # Retorna uma representação legível do grafo, mostrando os pesos.
        # Ex: "A: [('B', 5), ('C', 2)]\nB: [('A', 5)]"
        return '\n'.join([f"{no}: {vizinhos}" for no, vizinhos in self.adj.items()])

    def adicionar_aresta(self, n1, n2, peso=1, bidirecional=True): # Adicionado 'peso' com valor padrão 1
        # Garante que os nós existam no dicionário de adjacência
        if n1 not in self.adj:
            self.adj[n1] = []
        if n2 not in self.adj:
            self.adj[n2] = []

        # Adiciona a aresta de n1 para n2 com o peso
        # A lista agora armazena tuplas (nó_vizinho, peso)
        self.adj[n1].append((n2, peso))

        # Se for bidirecional, adiciona a aresta de n2 para n1 com o mesmo peso
        if bidirecional:
            self.adj[n2].append((n1, peso))

    def obter_vizinhos(self, n):
        # Retorna a lista de vizinhos de um nó (incluindo seus pesos).
        # Retorna uma lista vazia se o nó não existir.
        # Ex: Se 'A' tem vizinhos 'B' com peso 5 e 'C' com peso 2, retornaria [('B', 5), ('C', 2)]
        return self.adj.get(n, [])

    def obter_peso_aresta(self, n1, n2):
        """
        Retorna o peso da aresta entre n1 e n2.
        Retorna None se a aresta ou os nós não existirem.
        """
        if n1 not in self.adj:
            return None # n1 não existe

        # Procura por n2 na lista de adjacência de n1
        for vizinho, peso in self.adj[n1]:
            if vizinho == n2:
                return peso
        return None # Aresta não encontrada

    # Comum para DFS e BFS
    def __visitar(self, no):
        # Marca um nó como visitado e o adiciona à ordem de visita.
        self.__visitado[no] = 1
        self.ordem_visitados.append(no)

    # DFS - Busca em Profundidade
    def __dfs(self, n):
        # Implementação recursiva da Busca em Profundidade (DFS).
        self.__visitar(n)
        # Para grafos ponderados, 'obter_vizinhos' retorna tuplas (vizinho, peso).
        # Precisamos desempacotar isso para pegar apenas o 'vizinho'.
        for vizinho_info in self.obter_vizinhos(n):
            vizinho = vizinho_info[0] # Pega apenas o rótulo do vizinho
            # Verifica se o vizinho não foi visitado.
            if self.__visitado.get(vizinho, 0) == 0:
                self.__dfs(vizinho)

    def DFS(self, no_inicial):
        # Inicia a Busca em Profundidade (DFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós para cada nova execução de DFS.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        if no_inicial in self.adj:
            self.__dfs(no_inicial)
        return self.ordem_visitados
    # DFS - Busca em profundidade

    # BFS - Busca em largura
    def BFS(self, no_inicial):
        # Inicia a Busca em Largura (BFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        fila = []  # Usaremos uma lista como fila para a BFS

        if no_inicial not in self.adj:
            return self.ordem_visitados

        self.__visitar(no_inicial)
        fila.append(no_inicial)

        while fila:
            no_atual = fila.pop(0)  # Pega o primeiro nó da fila
            # Para grafos ponderados, 'obter_vizinhos' retorna tuplas (vizinho, peso).
            # Precisamos desempacotar isso para pegar apenas o 'vizinho'.
            for vizinho_info in self.obter_vizinhos(no_atual):
                vizinho = vizinho_info[0] # Pega apenas o rótulo do vizinho
                if self.__visitado.get(vizinho, 0) == 0:
                    self.__visitar(vizinho)
                    fila.append(vizinho)
        return self.ordem_visitados
    # BFS - Busca em largura

    # Algoritmo para encontrar o menor caminho de um nó determinado para todos os outros, em grafos sem pesos
    # Este método é o 'Single-Source Shortest Path' (SSSP) para grafos NÃO PONDERADOS (usando BFS)
    def __sssp_bfs(self, no_inicial, limite, funcao_juncao):
        """
        Função auxiliar para encontrar o menor caminho usando uma abordagem similar a BFS.
        Adaptada para usar rótulos de nós arbitrários. Esta versão NÃO USA OS PESOS
        das arestas. Ela encontra o caminho com o MENOR NÚMERO de arestas.
        """
        caminhos = {no_inicial: [no_inicial]} 
        fila = [(no_inicial, 0)] # (no, nível)
        
        while fila:
            no_atual, nivel = fila.pop(0)

            if limite is not None and nivel >= limite:
                continue

            # Para grafos ponderados, 'obter_vizinhos' retorna tuplas (vizinho, peso).
            # Para SSSP em grafos NÃO ponderados, só precisamos do rótulo do vizinho.
            for vizinho_info in self.obter_vizinhos(no_atual):
                vizinho = vizinho_info[0] # Pega apenas o rótulo do vizinho
                if vizinho not in caminhos: # Se o vizinho ainda não foi visitado para um caminho
                    caminhos[vizinho] = funcao_juncao(caminhos[no_atual], [vizinho])
                    fila.append((vizinho, nivel + 1))
        return caminhos
    
    def SSSP(self, no_inicial, limite=None):
        """
        Encontra os menores caminhos (em número de arestas) de um nó de partida para todos os nós alcançáveis.
        Este método é para grafos NÃO PONDERADOS ou quando os pesos não importam.
        """
        def funcao_juncao_padrao(caminho1, caminho2):
            return caminho1 + caminho2

        if no_inicial not in self.adj:
            return {} # Retorna um dicionário vazio se o nó de partida não existe

        return self.__sssp_bfs(no_inicial, limite, funcao_juncao_padrao)
    # Algoritmo para encontrar o menor caminho de um nó determinado para todos os outros, em grafos sem pesos


<img src=https://objectstorage.us-ashburn-1.oraclecloud.com/n/ida8x1uljntv/b/graph_images_ada/o/grafo_com_peso.png width=500>

In [ ]:
grafo_ponderado = GrafoLA()
grafo_ponderado.adicionar_aresta(1, 3, peso=4)
grafo_ponderado.adicionar_aresta(2, 3, peso=3)
grafo_ponderado.adicionar_aresta(3, 4, peso=7)
grafo_ponderado.adicionar_aresta(4, 5, peso=2)
grafo_ponderado.adicionar_aresta(4, 6, peso=9)
grafo_ponderado.adicionar_aresta(5, 6, peso=6)
grafo_ponderado.adicionar_aresta(5, 5, peso=0)

print(grafo_ponderado)

In [ ]:
grafo_ponderado.SSSP(1)

Notem que o SSSP indicaria que o menor caminho de 1 a 6 seria [1, 3, 4, 6] (custo 20), mas podemos ver que [1, 3, 2, 5, 6], embora utilize mais arestas, tem um custo menor (15). E agora?


____________________________

## Algoritmo de Dijkstra

Aqui vamos utilizar o algoritmo de Dijkstra por ser um dos mais famosos.

Este algoritmo calcula **a menor distância entre um nó e todos os outros nós no grafo**.

Considere o grafo a seguir:

<img src="https://www.codingame.com/servlet/fileservlet?id=14497257275137" width=300>

Vamos calcular o menor caminho entre o nó **C** e todos os outros nó do grafo!!

Durante o algoritmo, vamos marcar todos os nós **com a menor distância entre este nó e o nó C**.

A distância entre o nó C e ele mesmo é 0.

Inicialmente, a distância entre todos os nós é $\infty$.

A cada iteração, vamos também focar em um único nó, o **nó atual**, que será marcado por um ponto vermelho.

<img src="https://www.codingame.com/servlet/fileservlet?id=14497265537633" width=300>

Nós vamos também criar listas para os menores caminhos! Isso vai permitir termos não somente o comprimento dos menores caminhos entre os nós e **C**, como também o caminho em si!

Inicialmente, temos:

```
Caminho entre C e A = []
Caminho entre C e B = []
Caminho entre C e C = [C]
Caminho entre C e D = []
Caminho entre C e E = []
```

As listas serão alteradas toda vez que atualizarmos as distâncias mínimas, de modo que as listas finais expressarão o menor caminho!

Vamos ver o passo a passo:

### Nó atual: C

<img src="https://www.codingame.com/servlet/fileservlet?id=14497279927597" width=300>

<img src="https://www.codingame.com/servlet/fileservlet?id=14497284902206" width=300>

<img src="https://www.codingame.com/servlet/fileservlet?id=14497297264677" width=300>

<img src="https://www.codingame.com/servlet/fileservlet?id=14497301316895" width=300>

```
Caminho entre C e A = [C, A]
Caminho entre C e B = [C, B]
Caminho entre C e C = [C]
Caminho entre C e D = [C, D]
Caminho entre C e E = []
```

### Nó atual: A

<img src="https://www.codingame.com/servlet/fileservlet?id=14497311165233" width=300>

<img src="https://www.codingame.com/servlet/fileservlet?id=14497327460640" width=300>

```
Caminho entre C e A = [C, A]
Caminho entre C e B = [C, A, B]
Caminho entre C e C = [C]
Caminho entre C e D = [C, D]
Caminho entre C e E = []
```

### Nó atual: D

<img src="https://www.codingame.com/servlet/fileservlet?id=14497330975308" width=300>

```
Caminho entre C e A = [C, A]
Caminho entre C e B = [C, A, B]
Caminho entre C e C = [C]
Caminho entre C e D = [C, D]
Caminho entre C e E = [C, D, E]
```

### Nó atual: B

<img src="https://www.codingame.com/servlet/fileservlet?id=14497346742885" width=300>

```
Caminho entre C e A = [C, A]
Caminho entre C e B = [C, A, B]
Caminho entre C e C = [C]
Caminho entre C e D = [C, D]
Caminho entre C e E = [(C, B), E] = [C, A, B, E]
```

### Nó atual: E

<img src="https://www.codingame.com/servlet/fileservlet?id=14497350226741" width=300>

### FIM

<img src="https://www.codingame.com/servlet/fileservlet?id=14497361633811" width=300>

```
Caminho entre C e A = [C, A]
Caminho entre C e B = [C, A, B]
Caminho entre C e C = [C]
Caminho entre C e D = [C, D]
Caminho entre C e E = [C, A, B, E]
```

Esquematicamente, temos o algoritmo:


- 1: Marque o nó de origem com distância zero, e os demais nós com distância $\infty$

- 2: Atribua aos nós não visitados a menor entre as distâncias atuais entre o nó C

- 3: Para cada vizinho do nó C: adicione a distância atual de C com o peso da aresta conectando eles. Se for menor que a distância atual, mude a distância atual para este valor.

- 4: Marque o nó atual C como visitado.

- 5: Se ainda houver nós não visitados, volte para o passo 2.

In [ ]:
import math # Importar math para usar math.inf (infinito) para distâncias

class GrafoLA:
    def __init__(self):
        # self.adj agora mapeia rótulos de nós (inteiros ou strings)
        # para uma lista de tuplas (vizinho, peso).
        # Ex: {'A': [('B', 5), ('C', 2)], 'B': [('A', 5)]}
        self.adj = {}
        
        # Para controlar os nós visitados em algoritmos como Busca em Profundidade (DFS) e Busca em Largura (BFS),
        # usaremos um dicionário que mapeia o rótulo do nó para um status (0 para não visitado, 1 para visitado).
        self.__visitado = {}
        self.ordem_visitados = []

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        # Retorna uma representação legível do grafo, mostrando os pesos.
        # Ex: "A: [('B', 5), ('C', 2)]\nB: [('A', 5)]"
        return '\n'.join([f"{no}: {vizinhos}" for no, vizinhos in self.adj.items()])

    def adicionar_aresta(self, n1, n2, peso=1, bidirecional=True): # Adicionado 'peso' com valor padrão 1
        # Garante que os nós existam no dicionário de adjacência
        if n1 not in self.adj:
            self.adj[n1] = []
        if n2 not in self.adj:
            self.adj[n2] = []

        # Adiciona a aresta de n1 para n2 com o peso
        # A lista agora armazena tuplas (nó_vizinho, peso)
        self.adj[n1].append((n2, peso))

        # Se for bidirecional, adiciona a aresta de n2 para n1 com o mesmo peso
        if bidirecional:
            self.adj[n2].append((n1, peso))

    def obter_vizinhos(self, n):
        # Retorna a lista de vizinhos de um nó (incluindo seus pesos).
        # Retorna uma lista vazia se o nó não existir.
        # Ex: Se 'A' tem vizinhos 'B' com peso 5 e 'C' com peso 2, retornaria [('B', 5), ('C', 2)]
        return self.adj.get(n, [])

    def obter_peso_aresta(self, n1, n2):
        """
        Retorna o peso da aresta entre n1 e n2.
        Retorna None se a aresta ou os nós não existirem.
        """
        if n1 not in self.adj:
            return None # n1 não existe

        # Procura por n2 na lista de adjacência de n1
        for vizinho, peso in self.adj[n1]:
            if vizinho == n2:
                return peso
        return None # Aresta não encontrada

    # Comum para DFS e BFS
    def __visitar(self, no):
        # Marca um nó como visitado e o adiciona à ordem de visita.
        self.__visitado[no] = 1
        self.ordem_visitados.append(no)

    # DFS - Busca em Profundidade
    def __dfs(self, n):
        # Implementação recursiva da Busca em Profundidade (DFS).
        self.__visitar(n)
        # Para grafos ponderados, 'obter_vizinhos' retorna tuplas (vizinho, peso).
        # Precisamos desempacotar isso para pegar apenas o 'vizinho'.
        for vizinho_info in self.obter_vizinhos(n):
            vizinho = vizinho_info[0] # Pega apenas o rótulo do vizinho
            # Verifica se o vizinho não foi visitado.
            if self.__visitado.get(vizinho, 0) == 0:
                self.__dfs(vizinho)

    def DFS(self, no_inicial):
        # Inicia a Busca em Profundidade (DFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós para cada nova execução de DFS.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        if no_inicial in self.adj:
            self.__dfs(no_inicial)
        return self.ordem_visitados
    # DFS - Busca em profundidade

    # BFS - Busca em largura
    def BFS(self, no_inicial):
        # Inicia a Busca em Largura (BFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        fila = []  # Usaremos uma lista como fila para a BFS

        if no_inicial not in self.adj:
            return self.ordem_visitados

        self.__visitar(no_inicial)
        fila.append(no_inicial)

        while fila:
            no_atual = fila.pop(0)  # Pega o primeiro nó da fila
            # Para grafos ponderados, 'obter_vizinhos' retorna tuplas (vizinho, peso).
            # Precisamos desempacotar isso para pegar apenas o 'vizinho'.
            for vizinho_info in self.obter_vizinhos(no_atual):
                vizinho = vizinho_info[0] # Pega apenas o rótulo do vizinho
                if self.__visitado.get(vizinho, 0) == 0:
                    self.__visitar(vizinho)
                    fila.append(vizinho)
        return self.ordem_visitados
    # BFS - Busca em largura

    # Algoritmo para encontrar o menor caminho de um nó determinado para todos os outros, em grafos sem pesos
    # Este método é o 'Single-Source Shortest Path' (SSSP) para grafos NÃO PONDERADOS (usando BFS)
    def __sssp_bfs(self, no_inicial, limite, funcao_juncao):
        """
        Função auxiliar para encontrar o menor caminho usando uma abordagem similar a BFS.
        Adaptada para usar rótulos de nós arbitrários. Esta versão NÃO USA OS PESOS
        das arestas. Ela encontra o caminho com o MENOR NÚMERO de arestas.
        """
        caminhos = {no_inicial: [no_inicial]} 
        fila = [(no_inicial, 0)] # (no, nível)
        
        while fila:
            no_atual, nivel = fila.pop(0)

            if limite is not None and nivel >= limite:
                continue

            # Para grafos ponderados, 'obter_vizinhos' retorna tuplas (vizinho, peso).
            # Para SSSP em grafos NÃO ponderados, só precisamos do rótulo do vizinho.
            for vizinho_info in self.obter_vizinhos(no_atual):
                vizinho = vizinho_info[0] # Pega apenas o rótulo do vizinho
                if vizinho not in caminhos: # Se o vizinho ainda não foi visitado para um caminho
                    caminhos[vizinho] = funcao_juncao(caminhos[no_atual], [vizinho])
                    fila.append((vizinho, nivel + 1))
        return caminhos
    
    def SSSP(self, no_inicial, limite=None):
        """
        Encontra os menores caminhos (em número de arestas) de um nó de partida para todos os nós alcançáveis.
        Este método é para grafos NÃO PONDERADOS ou quando os pesos não importam.
        """
        def funcao_juncao_padrao(caminho1, caminho2):
            return caminho1 + caminho2

        if no_inicial not in self.adj:
            return {} # Retorna um dicionário vazio se o nó de partida não existe

        return self.__sssp_bfs(no_inicial, limite, funcao_juncao_padrao)
    # Algoritmo para encontrar o menor caminho de um nó determinado para todos os outros, em grafos sem pesos

    # DIJKSTRA (para Menor Caminho em Grafos Ponderados)
    def DIJKSTRA(self, no_inicial):
        """
        Encontra os menores caminhos e suas distâncias de um nó inicial para todos os outros
        em um grafo ponderado, usando o algoritmo de Dijkstra.
        Retorna um dicionário de distâncias e um dicionário de predecessores.
        """
        # Distâncias: {nó: distância_mínima_do_no_inicial}
        distancias = {no: math.inf for no in self.adj}
        distancias[no_inicial] = 0

        # Predecessores: {nó: nó_anterior_no_menor_caminho}
        predecessores = {no: None for no in self.adj}

        # Conjunto de nós já visitados/finalizados (com a menor distância encontrada)
        nos_visitados = set()

        # Fila de prioridade (simulada com lista, mas idealmente uma heapq para performance)
        # Elementos: (distância_atual_ao_no, nó)
        fila_prioridade = [(0, no_inicial)]

        while fila_prioridade:
            # Pega o nó com a menor distância atual da fila de prioridade
            # Para uma lista simples, isso é ineficiente (O(N) para min + pop)
            # Uma heap de min-prioridade (heapq) faria isso em O(logN)
            dist_atual, no_atual = fila_prioridade.pop(fila_prioridade.index(min(fila_prioridade)))

            # Se já visitamos este nó e encontramos um caminho mais curto antes, pule
            if no_atual in nos_visitados:
                continue
            
            nos_visitados.add(no_atual)

            # Explora os vizinhos do nó atual
            for vizinho_info in self.obter_vizinhos(no_atual):
                vizinho, peso_aresta = vizinho_info

                # Se o vizinho já foi visitado, pule
                if vizinho in nos_visitados:
                    continue

                # Calcula a nova distância potencial através do no_atual
                nova_distancia = dist_atual + peso_aresta

                # Se a nova distância for menor que a distância atualmente conhecida para o vizinho
                if nova_distancia < distancias[vizinho]:
                    distancias[vizinho] = nova_distancia
                    predecessores[vizinho] = no_atual
                    fila_prioridade.append((nova_distancia, vizinho)) # Adiciona/atualiza na fila

        # Formatar os caminhos reais
        caminhos_finais = {}
        for no_destino in self.adj:
            if distancias[no_destino] == math.inf:
                caminhos_finais[no_destino] = [] # Não alcançável
            else:
                caminho = []
                temp_no = no_destino
                while temp_no is not None:
                    caminho.insert(0, temp_no) # Insere no início para construir o caminho na ordem correta
                    temp_no = predecessores[temp_no]
                caminhos_finais[no_destino] = caminho

        return distancias, caminhos_finais
    # DIJKSTRA (para Menor Caminho em Grafos Ponderados)

In [ ]:
# FUNÇÃO DIJKSTRA(self, no_inicial):
#   """
#   Encontra os menores caminhos e suas distâncias de um nó inicial para todos os outros
#   em um grafo ponderado.
#   Retorna um dicionário de distâncias e um dicionário de caminhos.
#   """

#   // Inicialização
#   DEFINIR distancias = UM DICIONÁRIO ONDE CADA no EM self.adj TEM O VALOR DE INFINITO
#   DEFINIR distancias[no_inicial] = 0 // A distância do nó inicial para ele mesmo é zero

#   DEFINIR predecessores = UM DICIONÁRIO ONDE CADA no EM self.adj TEM O VALOR NULO
#   // predecessores[no] guardará o nó que veio antes de 'no' no menor caminho

#   DEFINIR nos_visitados = UM CONJUNTO VAZIO
#   // nos_visitados guardará os nós para os quais já encontramos a menor distância final

#   DEFINIR fila_prioridade = UMA LISTA CONTENDO UMA TUPLA (0, no_inicial)
#   // A fila de prioridade armazena tuplas (distância_atual_ao_no, nó)
#   // O nó com a menor distância atual será sempre o próximo a ser processado

#   // Loop Principal de Dijkstra
#   ENQUANTO fila_prioridade NÃO ESTÁ VAZIA:
#     // Pega o nó da fila_prioridade com a MENOR distância atual
#     // (Simulação de uma fila de prioridade: encontra o mínimo e o remove)
#     DEFINIR dist_atual, no_atual = REMOVER ELEMENTO COM MENOR DISTÂNCIA DA fila_prioridade

#     // SE no_atual JÁ ESTÁ EM nos_visitados:
#     //   CONTINUAR // Se já processamos este nó (e sua menor distância já foi finalizada), pule
#     // FIM SE
#     // CORREÇÃO: O Python faz isso de forma menos eficiente com lista. A lógica é: se a distância já é maior, pule
#     SE dist_atual > distancias[no_atual]:
#       CONTINUAR // Já encontramos um caminho mais curto para este nó
#     FIM SE
    
#     ADICIONAR no_atual AO nos_visitados

#     // Explora os vizinhos do nó_atual
#     PARA CADA vizinho_info EM self.obter_vizinhos(no_atual):
#       DEFINIR vizinho, peso_aresta = vizinho_info // Desempacota a tupla (vizinho, peso)

#       // SE vizinho ESTÁ EM nos_visitados:
#       //   CONTINUAR // Se o vizinho já foi visitado, pule
#       // FIM SE

#       // Calcula a distância potencial para o vizinho através do no_atual
#       DEFINIR nova_distancia = dist_atual + peso_aresta

#       // SE nova_distancia É MENOR QUE distancias[vizinho]:
#       //   DEFINIR distancias[vizinho] = nova_distancia
#       //   DEFINIR predecessores[vizinho] = no_atual
#       //   ADICIONAR TUPLA (nova_distancia, vizinho) À fila_prioridade
#       // FIM SE
#       // CORREÇÃO: A condição de "não visitado" já está implícita pela verificação de "dist_atual > distancias[no_atual]"
#       SE nova_distancia < distancias[vizinho]:
#         DEFINIR distancias[vizinho] = nova_distancia
#         DEFINIR predecessores[vizinho] = no_atual
#         ADICIONAR TUPLA (nova_distancia, vizinho) À fila_prioridade
#       FIM SE
#     FIM PARA
#   FIM ENQUANTO

#   // Reconstrução dos Caminhos Finais
#   DEFINIR caminhos_finais = UM DICIONÁRIO VAZIO
#   PARA CADA no_destino EM self.adj:
#     SE distancias[no_destino] É IGUAL A INFINITO:
#       DEFINIR caminhos_finais[no_destino] = UMA LISTA VAZIA // Nó não alcançável
#     SENÃO:
#       DEFINIR caminho = UMA LISTA VAZIA
#       DEFINIR temp_no = no_destino
#       ENQUANTO temp_no NÃO É NULO:
#         INSERIR temp_no NO INÍCIO DE caminho // Constrói o caminho na ordem correta (do início ao destino)
#         DEFINIR temp_no = predecessores[temp_no]
#       FIM ENQUANTO
#       DEFINIR caminhos_finais[no_destino] = caminho
#     FIM SE
#   FIM PARA

#   RETORNAR distancias, caminhos_finais
# FIM FUNÇÃO

In [ ]:
import math

print("--- Grafo Ponderado com Cidades e Distâncias ---")
grafo_cidades = GrafoLA()

grafo_cidades.adicionar_aresta("Porto Alegre", "Gramado", peso=120)
grafo_cidades.adicionar_aresta("Porto Alegre", "Caxias do Sul", peso=130)
grafo_cidades.adicionar_aresta("Gramado", "Canela", peso=8)
grafo_cidades.adicionar_aresta("Gramado", "Nova Petrópolis", peso=35)
grafo_cidades.adicionar_aresta("Caxias do Sul", "Bento Gonçalves", peso=40)
grafo_cidades.adicionar_aresta("Bento Gonçalves", "Garibaldi", peso=15)
grafo_cidades.adicionar_aresta("Nova Petrópolis", "Caxias do Sul", peso=60) # Nova conexão

print("Grafo:")
print(grafo_cidades)

print("\n--- Utilizando Dijkstra para Menores Caminhos Ponderados ---")
distancias, caminhos = grafo_cidades.DIJKSTRA("Porto Alegre")

print(f"\nMenores distâncias a partir de 'Porto Alegre':")
for no, dist in distancias.items():
    print(f"  Para {no}: {dist if dist != math.inf else 'Inalcançável'}")

print(f"\nMenores caminhos a partir de 'Porto Alegre':")
for no, caminho in caminhos.items():
    print(f"  Para {no}: {' -> '.join(map(str, caminho)) if caminho else 'Inalcançável'}")